In [12]:
import pandas as pd
import seaborn as sns
from matplotlib import font_manager
import matplotlib.pyplot as plt
import plotly.express as px
import os

In [13]:
#Prepare Thai font for matplotlib
#ref: https://gist.github.com/korakot/3021b0027607b8102d98faee3a7e156a
font_dirs = ["../assets/th-sarabun-new"]

font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
print(font_files)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 18.0
plt.rcParams['ytick.labelsize'] = 18.0

['/Users/pmr/Workspace/9geek_data/thai_local_revenue_datahack/assets/th-sarabun-new/THSarabunNew BoldItalic.ttf', '/Users/pmr/Workspace/9geek_data/thai_local_revenue_datahack/assets/th-sarabun-new/THSarabunNew.ttf', '/Users/pmr/Workspace/9geek_data/thai_local_revenue_datahack/assets/th-sarabun-new/THSarabunNew Bold.ttf', '/Users/pmr/Workspace/9geek_data/thai_local_revenue_datahack/assets/th-sarabun-new/THSarabunNew Italic.ttf']


In [14]:
data_dir = "../data/"
data_processed_dir = "../data_processed/"

In [15]:
data_df = pd.read_csv(os.path.join(data_processed_dir, 'local_revenue_61to65_processed.csv'))
data_df.tail(n=1)

,ลำดับ,ชื่อท้องถิ่น,อำเภอ,จังหวัด,พื้นที่ (ตร.กม.),ประชากร,ภาษีโรงเรือนและที่ดิน,ภาษีบำรุงท้องที่,ภาษีป้าย,อากรการฆ่าสัตว์,...,ค่าธรรมเนียมน้ำบาดาล,ค่าธรรมเนียมการจดทะเบียนสิทธิและนิติกรรมตามประมวลกฎหมายที่ดิน,ภาษีจัดสรรอื่น,รายได้รัฐจัดสรร,เงินอุดหนุนทั่วไป,เงินอุดหนุนเฉพาะกิจ,รายได้จากเงินอุดหนุน,รายได้รวม,data_year,ภาษีที่ดินและสิ่งปลูกสร้าง
32602,"7,850",อบต.หนองบก,เหล่าเสือโก้ก,อุบลราชธานี,50.0,7823.0,NaN,NaN,100360.0,NaN,...,4300.0,1521658.0,NaN,23947786.78,25780372.0,NaN,25780372.0,50392524.32,2565,288207.14


There are only 1199 rows of data in 2563 while other years have 7850 - 7852 rows.
It seems that data for 2563 is incomplete. Let's skip it.

In [16]:
selected_years = [2561, 2562, 2564, 2565]
data_df = data_df[data_df['data_year'].isin(selected_years)]

### 1. รายได้รวมแต่ละปี

In [17]:
year_column = 'data_year' 
total_revenue_column = 'รายได้รวม' 

yearly_revenue = data_df.groupby(year_column)[total_revenue_column].sum().reset_index()

fig = px.bar(yearly_revenue, x='data_year', y='รายได้รวม',
             labels={'รายได้รวม': 'Total Revenue (THB)', 'data_year': 'Year'},
             title='Trend of Total Revenue Over Years (in Millions)')

# Updating y-axis format to display in millions with 'M' suffix
fig.update_yaxes(tickformat=".0f", tickvals=[100000000000, 200000000000, 300000000000, 400000000000, 500000000000, 600000000000, 700000000000],
                 ticktext=["100,000 M", "200,000 M", "300,000 M", "400,000 M", "500,000 M", "600,000 M", "700,000 M"])

# Show the figure
fig.show()

In [18]:
temp_yearly_rev = yearly_revenue.copy()
temp_yearly_rev['รายได้รวม'] = temp_yearly_rev['รายได้รวม'].apply(lambda x: f'{int(x/1000000)} M')
temp_yearly_rev.sort_values(by='รายได้รวม', ascending=False)

,data_year,รายได้รวม
3,2565,696365 M
1,2562,671426 M
2,2564,658073 M
0,2561,636573 M


The local revenue in Thailand showed an overall increase from 2561 to 2565, with a temporary dip in 2564. The revenue grew from 636,573 million THB in 2561 to 696,365 million THB in 2565, indicating positive growth despite the brief decline in 2564.

### 2. ภาษีแต่ละปี

In [19]:
#col with ภาษี
tax_columns = [col for col in data_df.columns if 'ภาษี' in col and col != 'ภาษีสุรา']
tax_columns

['ภาษีโรงเรือนและที่ดิน',
 'ภาษีบำรุงท้องที่',
 'ภาษีป้าย',
 'ภาษีบำรุง อบจ. จากสถานค้าปลีกยาสูบ',
 'ภาษีบำรุงจากสถานค้าปลีกน้ำมัน',
 'ภาษีและค่าธรรมเนียมรถยนต์หรือล้อเลื่อน',
 'ภาษีมูลค่าเพิ่มที่จัดเก็บตามประมวลรัษฎากร 5%(อบจ)',
 'ภาษีมูลค่าเพิ่มตามกฎหมายท้องถิ่น',
 'ภาษีมูลค่าเพิ่มตาม พ.ร.บ. กำหนดแผนฯ',
 'ภาษีธุรกิจเฉพาะ',
 'ภาษีสรรพสามิต',
 'ภาษีการพนัน',
 'ภาษียาสูบ',
 'ภาษีจัดสรรอื่น',
 'ภาษีที่ดินและสิ่งปลูกสร้าง']

In [20]:
year_wise_tax_trends = data_df.groupby('data_year')[tax_columns].sum()
year_wise_tax_trends

,ภาษีโรงเรือนและที่ดิน,ภาษีบำรุงท้องที่,ภาษีป้าย,ภาษีบำรุง อบจ. จากสถานค้าปลีกยาสูบ,ภาษีบำรุงจากสถานค้าปลีกน้ำมัน,ภาษีและค่าธรรมเนียมรถยนต์หรือล้อเลื่อน,ภาษีมูลค่าเพิ่มที่จัดเก็บตามประมวลรัษฎากร 5%(อบจ),ภาษีมูลค่าเพิ่มตามกฎหมายท้องถิ่น,ภาษีมูลค่าเพิ่มตาม พ.ร.บ. กำหนดแผนฯ,ภาษีธุรกิจเฉพาะ,ภาษีสรรพสามิต,ภาษีการพนัน,ภาษียาสูบ,ภาษีจัดสรรอื่น,ภาษีที่ดินและสิ่งปลูกสร้าง
data_year,,,,,,,,,,,,,,,
2561,3.338509e+10,9.392583e+08,3.424266e+09,2.326037e+09,1.045866e+09,3.736208e+10,1.319434e+10,6.153644e+10,1.123217e+11,5.936046e+09,5.268305e+10,48197157.98,213697.03,1.139799e+08,0.000000e+00
2562,3.555931e+10,9.680514e+08,3.609563e+09,2.544459e+09,1.075313e+09,3.751251e+10,1.420852e+10,5.557129e+10,1.142868e+11,6.118598e+09,5.726606e+10,23981376.76,197118.08,2.964618e+08,0.000000e+00
2564,8.587179e+08,7.975260e+07,4.441548e+09,2.368058e+09,1.020515e+09,3.799507e+10,1.288850e+10,6.034384e+10,1.110617e+11,5.366594e+09,5.279930e+10,73271.68,173140.13,1.579494e+08,4.105412e+09
2565,8.970875e+08,3.040190e+07,4.550797e+09,1.974212e+09,1.057750e+09,3.902083e+10,1.501519e+10,7.254930e+10,1.272110e+11,5.766956e+09,4.824880e+10,103288.25,261.90,1.248215e+08,3.518694e+10


In [21]:
year_wise_tax_trends.columns

Index(['ภาษีโรงเรือนและที่ดิน', 'ภาษีบำรุงท้องที่', 'ภาษีป้าย',
       'ภาษีบำรุง อบจ. จากสถานค้าปลีกยาสูบ', 'ภาษีบำรุงจากสถานค้าปลีกน้ำมัน',
       'ภาษีและค่าธรรมเนียมรถยนต์หรือล้อเลื่อน',
       'ภาษีมูลค่าเพิ่มที่จัดเก็บตามประมวลรัษฎากร 5%(อบจ)',
       'ภาษีมูลค่าเพิ่มตามกฎหมายท้องถิ่น',
       'ภาษีมูลค่าเพิ่มตาม พ.ร.บ. กำหนดแผนฯ', 'ภาษีธุรกิจเฉพาะ',
       'ภาษีสรรพสามิต', 'ภาษีการพนัน', 'ภาษียาสูบ', 'ภาษีจัดสรรอื่น',
       'ภาษีที่ดินและสิ่งปลูกสร้าง'],
      dtype='object')

In [22]:
melted_data_select_col = tax_columns
melted_data = year_wise_tax_trends[melted_data_select_col].reset_index().melt(id_vars=['data_year'], var_name='Metric', value_name='Value')

# Converting the 'Value' column to millions (THB)
melted_data['Value'] /= 1000000  # Convert to millions

fig = px.bar(melted_data, x='data_year', y='Value', color='Metric', barmode='group',
             labels={'Value': 'Value (Millions THB)', 'data_year': 'Year'},
             title='Year-wise Trends of Various Metrics (in Millions)')

fig.show()

### Summary for Each Type of Tax (2561 - 2565)

1. **ภาษีโรงเรือนและที่ดิน (Land and House Tax):** Fluctuated significantly over the years.
&nbsp;
2. **ภาษีบำรุงท้องที่ (Local Maintenance Tax):** Exhibited a decreasing trend.
&nbsp;
3. **ภาษีป้าย (Signage Tax):** Showed consistent growth, indicating increased revenue from advertising.
&nbsp;
4. **ภาษีบำรุง อบจ. (Taxes from Tobacco/Oil Retail):** Relatively stable over the years.
&nbsp;
5. **ภาษีและค่าธรรมเนียมรถยนต์ (Vehicle-Related Taxes and Fees):** Maintained stable revenue.
&nbsp;
6. **ภาษีมูลค่าเพิ่ม (VAT) Collections:** Saw a significant increase, especially in later years.
&nbsp;
7. **ภาษีธุรกิจเฉพาะ (Specific Business Tax):** Varied year to year but showed growth.
&nbsp;
8. **ภาษีสรรพสามิต (Excise Tax):** Generally stable with slight fluctuations.
&nbsp;
9. **ภาษียาสูบ (Tobacco Tax):** Had minimal changes over the years.
&nbsp;
10. **ภาษีที่ดินและสิ่งปลูกสร้าง (Land and Construction Tax):** Introduced in 2564, showing a substantial figure by 2565.

"ภาษีโรงเรือน และที่ดิน" มีการจัดเก็บตาม พระราชบัญญัติภาษีโรงเรือนและที่ดิน พ.ศ.2475 และได้ยกเลิกไปเมื่อวันที่ 10 มีนาคม 2562 ตามพระราชบัญญัติภาษีที่ดินและสิ่งปลูกสร้าง พ.ศ. 2562

"ภาษีที่ดิน และสิ่งปลูกสร้าง" จัดเก็บตามพระราชบัญญัติภาษีที่ดินและสิ่งปลูกสร้าง พ.ศ. 2562
"มีวัตถุประสงค์เพื่อกระตุ้นให้เกิดการใช้ประโยชน์ที่ดิน หรือพูดง่าย ๆ คือรัฐไม่ต้องการให้ที่ดินถูกทิ้งร้างโดยไม่เกิดการพัฒนา"
&nbsp;
Reference: 
- [Workpoint Today](https://workpointtoday.com/saving-guru-16/)
- [Thansettakij](https://www.thansettakij.com/real-estate/566638)

Over the period from 2561 to 2565, Thai local governments experienced varied trends in different revenue streams. The **Value Added Tax (VAT) collections** ('ภาษีมูลค่าเพิ่ม'), particularly from พ.ร.บ. กำหนดแผนฯ, emerged as the highest income source, showing a marked increase over the years. In contrast, the **Local Maintenance Tax ('ภาษีบำรุงท้องที่')


### 2. รายได้รวมของแต่ละจังหวัด